# Introduction to Property Value Management (PVMT)

This notebook present the current capabilities of the PVMT extension for py-capellambse.
We will see how to display all defined property value domains, their groups and the
available properties.
Further we'll see how to access applied property values from ModelObjects. This is demonstrated on the list of LogicalActors.

In [1]:
import capellambse


path_to_model = "../tests/data/melodymodel/5_2/Melody Model Test.aird"
model = capellambse.MelodyModel(path_to_model, jupyter_untrusted=True)

You can investigate all defined PVMT domains, their groups and properties:

In [2]:
darkmagic = model.property_value_packages[0].packages[0]
display(darkmagic)
for grp in darkmagic.groups:
    display(grp.name)
    display(grp.properties)
    display(grp.scope)

constraints,(Empty list)
description,None
diagrams,(Empty list)
enumeration_property_types,"EnumerationPropertyType ""DarkMagicPowerEnum"" (2644e99b-72c8-42f0-9bba-5141dda592e9)"
groups,"PropertyValueGroup ""Power"" (1e241678-282c-46f3-abbd-e0cf34e743ee)PropertyValueGroup ""Power Level"" (a0cb5a23-955e-43d6-a633-2c4e66991364)"
name,DarkMagic
packages,(Empty list)
parent,"PropertyValuePkg ""EXTENSIONS"" (fdbeb3b8-e1e2-4c6d-aff4-bccb1b9f437f)"
progress_status,NOT_SET
properties,"StringPropertyValue ""version"": 1.0.0 (6b9fc2f4-48c4-49d1-96c2-0c082d64ec83)"
pvmt,<capellambse.extensions.pvmt.PropertyValueProxy object at 0x7f87238a3fd0>


'Power'

[0] <FloatPropertyValue 'Min' (4c5eba40-fcc3-480f-b88f-e375dadc00e5)>
[1] <FloatPropertyValue 'Max' (196467a2-ecee-4f35-8987-6a898cd34616)>

['LOGICAL', 'SYSTEM']

'Power Level'

[0] <EnumerationPropertyValue 'Skill' (7367c731-aabb-4807-9d74-7ef2c6482ec9)>

'LOGICAL'

The PropertyValueGroup has a scope attribute in which 3 types of optional rules can be defined for it. These 3 types are:
- `ARCHITECTURE`,
- `CLASS` and
- `PROP`

where the `ARCHITECTURE` rule controls on which architecture layer this group is valid, the `CLASS` rule limits the class type of `ModelObject`s this group can be applied to and the `PROP` rule grants the option to declare property based rules tied to the property type. Currently the `.scope` attribute only returns the `ARCHITECTURE` rule. For inspection of the whole scope access `.description`.

In [3]:
darkmagic.groups[0].description

Markup('[CLASS]http://www.polarsys.org/capella/core/la/5.0.0/LogicalComponent[/CLASS]\n[ARCHITECTURE]LOGICAL;SYSTEM[/ARCHITECTURE]')

## Access PVMT property values

The ground rule is that you can access PVMT via the `.pvmt` attribute on literally everything you can find in a model. Then the pattern is `domain.group.subgroup.[moresubgroups...].property` from which the pvmt can be indexed with:

Let's check Dumbledore's dark magic power level:

In [4]:
dumbledore = model.la.all_components.by_name("Prof. A. P. W. B. Dumbledore")
dumbledore.pvmt["DarkMagic.Power Level.Skill"]

'Mighty'

In our test model we have more famous wizzards. Let us define a function to collect all property values and load them into a pandas.DataFrame:

In [5]:
import pandas as pd


def get_darkmagic_dataframe(model: capellambse.MelodyModel) -> pd.DataFrame:
    """Return a DataFrame with the darkmagic power for all LogicalActors."""
    return pd.DataFrame(
        [
            {
                "Minimum Power": actor.pvmt["DarkMagic.Power.Min"],
                "Maximum Power": actor.pvmt["DarkMagic.Power.Max"],
                "Skill level": actor.pvmt["DarkMagic.Power Level.Skill"],
            } for actor in model.la.all_actors
        ], index=(actor.name for actor in model.la.all_actors)
    )


get_darkmagic_dataframe(model)

,Minimum Power,Maximum Power,Skill level
Voldemort,NaN,10000.0,TheOneWhoShallNotBeNamed
Prof. A. P. W. B. Dumbledore,800.0,1600.0,Mighty
R. Weasley,50.0,150.0,Apprentice
Prof. S. Snape,200.0,450.0,Mighty
Harry J. Potter,300.0,500.0,Apprentice


## Writing Property values

### How we want it to be

In [7]:
dumbledore.pvmt["DarkMagic.Power.Min"] = 1000.0

TypeError: 'PropertyValueProxy' object does not support item assignment

### How it works now

In [11]:
from capellambse import pvmt

pvext = pvmt.load_pvmt_from_model(model._loader)
darkmagic_power = pvext.get_element_pv(dumbledore._element, "DarkMagic.Power")
darkmagic_power["Min"] -= 100
darkmagic_power["Max"] -= 100

In [10]:
get_darkmagic_dataframe(model)

,Minimum Power,Maximum Power,Skill level
Voldemort,NaN,10000.0,TheOneWhoShallNotBeNamed
Prof. A. P. W. B. Dumbledore,600.0,1400.0,Exceptional
R. Weasley,50.0,150.0,Apprentice
Prof. S. Snape,200.0,450.0,Mighty
Harry J. Potter,300.0,500.0,Apprentice


### Setting a new Enum property

This should fail if set to a value for which an `EnumPropertyLiteral` doesn't exist yet:

In [12]:
darkmagic_power_level = pvext.get_element_pv(dumbledore._element, "DarkMagic.Power Level")
darkmagic_power_level["Skill"] = "Exceptional"

or create it automatically(?):

In [19]:
plvl_enum = darkmagic.enumeration_property_types[0]
exceptional_literal = plvl_enum.literals.create(name="Exceptional")
exceptional_literal.values

[]

In [20]:
plvl_enum.literals[0].values

[0] <StringPropertyValue '__LITERAL__' (6b6a788d-acf7-460c-b150-642276ca8acb)>

TODO: Creation of `EnumerationPropertyLiteral` is missing child element `StringPropertyValue` with name `__LITERAL__`. Is this needed or useless duplication of dead code?